# Parallel Simulation

Parallel simulation is a powerful technique to accelerate the training process by leveraging GPU capabilities. Instead of simulating only one robot or environment at a time, we can run multiple environments in parallel within the same scene.

## What You Will Learn

Hands-on with parallel simulation code and visualize parallel execution.

In [1]:
# Suppress warning messages for clearer output
import warnings
import os

os.environ["TI_LOG_LEVEL"] = "error"
warnings.filterwarnings("ignore")

## Init and Create a Scene

Just like previous labs. we create and init a scene in the beginning.

In [2]:
import numpy as np
import genesis as gs

########################## init ##########################
gs.init(backend=gs.vulkan, theme='light')

########################## create a scene ##########################
scene = gs.Scene(
    viewer_options=gs.options.ViewerOptions(
        camera_pos=(0.0, -2, 1.5),
        camera_lookat=(0.0, 0.0, 0.5),
        camera_fov=40,
        max_FPS=200,
    ),
    rigid_options=gs.options.RigidOptions(
        enable_joint_limit=False,
    ),
    show_viewer=False
)

[Genesis] [10:34:49] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [10:34:49] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [10:34:49] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [10:34:49] [INFO] Consider setting 'performance_mode=True' in production to maximise runtime speed, if significantly increasing compilation time is not a concern.
[Genesis] [10:34:49] [INFO] Running on [AMD Radeon Graphics] with backend gs.vulkan. Device memory: 60.75 GB.
[Genesis] [10:34:49] [INFO] 🚀 Genesis initialized. 🔖 version: 0.3.3, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'light'.
[Genesis] [10:34:49] [INFO] Scene <397158a> created.


## Add Entities

Just like what we did in previous labs, we add a **plane**, an **arm**, and a **camera** to the scene, and then build the scene.

In [3]:
########################## entities ##########################
plane = scene.add_entity(
    gs.morphs.Plane(),
)
robot = scene.add_entity(
    gs.morphs.MJCF(file="xml/franka_emika_panda/panda.xml"),
)

cam = scene.add_camera(
    res=(640, 480),
    pos=(6, 4, 2),
    lookat=(0, 0, 0.2),
    fov=30,
)

[Genesis] [10:35:17] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <1c869ef>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [10:35:17] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <c3b0391>, morph: <gs.morphs.MJCF(file='/opt/conda/envs/py_3.12/lib/python3.12/site-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [10:35:18] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [10:35:18] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias for joint `finger_joint1`.
[Genesis] [10:35:18] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [10:35:18] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias for joint `finger_joint2`.
[Genesis] [10:35:18] [INFO] Applying o

## Build the Scene

In Genesis, enabling scene-level parallelism is straightforward. When building your scene, you simply add the parameter `n_envs` to specify how many environments you want. To align with terminology commonly used in machine learning literature, we will also use the term `batching` to refer to parallelization.

In [4]:
########################## build ##########################
n_envs = 9
scene.build(n_envs=n_envs, env_spacing=(1.0, 1.0))

[Genesis] [10:35:38] [INFO] Building scene <397158a>...
[Genesis] [10:35:38] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [10:35:38] [WARNING] Constraint solver time constant should be greater than 2*substep_dt. timeconst is changed from `0.005` to `0.02`). Decrease simulation timestep or increase timeconst to avoid altering the original value.
[Genesis] [10:35:39] [INFO] Compiling simulation kernels...
[Genesis] [10:35:43] [INFO] Building visualizer...


amdgpu: os_same_file_description couldn't determine if two DRM fds reference the same file description.
If they do, bad things may happen!


## Controlling Robots in Batched Environments

In earlier tutorials, we used APIs such as franka.control_dofs_position() to control a single robot.
With parallel simulation, the same API can be extended naturally: simply add a batch dimension to control multiple robots at once.

In [5]:
import logging
from tqdm import tqdm

# Set logger to warning to avoid log info.
gs.logger._logger.setLevel(logging.WARNING)

rgb, depth, segmentation, normal = cam.render(rgb=True, depth=True, segmentation=True, normal=True)
cam.start_recording()

target_quat = np.tile(np.array([0, 1, 0, 0]), [n_envs, 1])  # pointing downwards
center = np.tile(np.array([0.4, -0.2, 0.25]), [n_envs, 1])
angular_speed = np.random.uniform(-10, 10, n_envs)
r = 0.1

ee_link = robot.get_link("hand")

for i in tqdm(range(1000), ncols=100):
    target_pos = np.zeros([n_envs, 3])
    target_pos[:, 0] = center[:, 0] + np.cos(i / 360 * np.pi * angular_speed) * r
    target_pos[:, 1] = center[:, 1] + np.sin(i / 360 * np.pi * angular_speed) * r
    target_pos[:, 2] = center[:, 2]
    target_q = np.hstack([target_pos, target_quat])

    q = robot.inverse_kinematics(
        link=ee_link,
        pos=target_pos,
        quat=target_quat,
        rot_mask=[False, False, True],  # for demo purpose: only restrict direction of z-axis
    )

    robot.set_qpos(q)
    scene.step()
    cam.render()
    
cam.stop_recording(save_to_filename="Videos/video_04.mp4", fps=60)

100%|███████████████████████████████████████████████████████████| 1000/1000 [00:46<00:00, 21.58it/s]


## Show the video

You will see nine robotic arms moving at different speeds and angles. This is because we set `n_envs` to nine and assigned each of them a random `angular_speed`.

In [6]:
from IPython.display import Video
Video(url="Videos/video_04.mp4")

# Congratulations! 

After completing these four labs, you now have a basic understanding of how to create and control a virtual robotic arm in Genesis. You’ve also learned how to plan a pick-and-place path for a block and how to perform parallel simulations within a scene.

If you’re interested in running Genesis on AMD machines, here are some useful references:

* **Genesis GitHub:** [https://github.com/Genesis-Embodied-AI/Genesis](https://github.com/Genesis-Embodied-AI/Genesis)
* **Train a Unitree Dog on Genesis:** [https://github.com/JingXunLin/Genesis_Go2](https://github.com/JingXunLin/Genesis_Go2)

See you in the next wrokshop!
